In [1]:
import yaml
import sys
import numpy as np
sys.path.insert(0, '/home/btech/mayank.kumar/Hritik/Multi-Task')
import pandas as pd
from sklearn.preprocessing import StandardScaler
from transformers import AdamW, get_linear_schedule_with_warmup, RobertaModel
from PolymerSmilesTokenization import PolymerSmilesTokenizer
from multiTask_dataset import MultiTask_Dataset
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from copy import deepcopy
from torch.utils.data import DataLoader

/home/btech/mayank.kumar/miniconda3/envs/TransPolymer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = pd.read_csv('../data/multiTask.csv')
data = data.sample(frac=1.0, random_state=42)
data.reset_index(drop=True, inplace=True)
train_data , test_data = train_test_split(data ,random_state=42, test_size=0.20, shuffle=False)

PretrainedModel = RobertaModel.from_pretrained('../ckpt/pretrain.pt')
tokenizer = PolymerSmilesTokenizer.from_pretrained("roberta-base", max_len=411)
PretrainedModel.config.hidden_dropout_prob = 0.1
PretrainedModel.config.attention_probs_dropout_prob = 0.1
max_token_len = 411

Some weights of the model checkpoint at ../ckpt/pretrain.pt were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../ckpt/pretrain.pt and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer clas

In [3]:
train_data

,smiles,Eea,Egb,Egc,Ei
0,*NC(=S)NC(=S)*,-99.0000,-99.0000,2.8537,-99.0000
1,*Oc1ccc(Oc2ccc(OC(*)=O)cc2)cc1,-99.0000,-99.0000,4.7541,-99.0000
2,*NCC(F)(F)C(*)(F)F,0.6587,7.8675,-99.0000,8.2676
3,*CCCCCCCCCCOC(=O)CC(C)(C)CC(=O)O*,-99.0000,-99.0000,6.7584,-99.0000
4,*CCCCCCNC(=O)C(CCCCCCCCCCCCCCC)C(=O)N*,-99.0000,-99.0000,5.9375,-99.0000
...,...,...,...,...,...
2967,*CC(CO*)(CS(=O)(=O)CCCC)CS(=O)(=O)CCCC,-99.0000,-99.0000,5.4465,-99.0000
2968,*CCCCCCCCCCNC(=O)CCP(C)(=O)CCC(=O)N*,-99.0000,-99.0000,5.8056,-99.0000
2969,*Oc1c(-c2ccccc2)cc(*)cc1-c1cccc(C)c1,-99.0000,-99.0000,3.8976,-99.0000
2970,*CCC(=O)OC(=O)CCC1CCC(*)O1,-99.0000,-99.0000,5.5098,-99.0000


In [4]:
test_data

,smiles,Eea,Egb,Egc,Ei
2972,*Nc1ccc(C(C)(C)c2ccc(NC(=O)c3cc(C(*)=O)cc(N4C(...,-99.0,-99.0,3.5748,-99.0
2973,*c1cccc(C(F)(F)C(F)(F)C(*)(F)F)c1,-99.0,-99.0,5.5439,-99.0
2974,*CCN(*)C(=O)c1ccc(F)cc1,-99.0,-99.0,5.1588,-99.0
2975,*c1ccc2cc(N(*)c3ccccc3)ccc2c1,-99.0,-99.0,2.8541,-99.0
2976,*Nc1cc(C)c(*)cc1C,-99.0,-99.0,3.9275,-99.0
...,...,...,...,...,...
3711,*CCCCCOC(=O)C(=O)O*,-99.0,-99.0,5.5658,-99.0
3712,*CCCCCCCCCCCCNC(=O)CCCCC(=O)N*,-99.0,-99.0,5.5516,-99.0
3713,*c1ccc(Oc2ccc(-c3cnc4ccc(-c5ccc6ncc(*)nc6c5)cc...,-99.0,-99.0,3.2298,-99.0
3714,*CCOCCOC(=O)NC(=CC=C1(C))C=C1NC(=O)O*,-99.0,-99.0,5.0931,-99.0


In [5]:
import torch
from torch.utils.data import Dataset
import numpy as np
from rdkit import Chem
from copy import deepcopy
import torch.nn as nn
import pandas as pd


class MultiTask_Dataset(Dataset):
    def __init__(self, dataset, tokenizer, max_token_len):
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.max_token_len = max_token_len
    
    def __len__(self):
        self.len = len(self.dataset)
        return self.len

    def __getitem__(self, i):
        data_row = self.dataset.iloc[i]

        smile = data_row[0]

        encoding = self.tokenizer(
            str(smile),
            add_special_tokens=True,
            max_length=self.max_token_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        props = []
        loss_mask = []

        for i in range(1,5):
            props.append(data_row[i])
            if(data_row[i]==-99):
                loss_mask.append(0)
            else:
                loss_mask.append(1)

        props = torch.tensor(props)
        loss_mask = torch.tensor(loss_mask)

        return dict(
            input_ids=encoding["input_ids"].flatten(),
            attention_mask=encoding["attention_mask"].flatten(),
            smile = smile,
            smile_len = len(smile)+2,
            props=props,
            loss_mask = loss_mask
        )
    
class MultiTask_DataAugmentation:
    def __init__(self, aug_indicator):
        super(MultiTask_DataAugmentation, self).__init__()
        self.aug_indicator = aug_indicator

    """Rotate atoms to generate more SMILES"""
    def rotate_atoms(self, li, x):
        return (li[x % len(li):] + li[:x % len(li)])

    """Generate SMILES"""
    def generate_smiles(self, smiles, prop):
        smiles_list = []
        try:
            mol = Chem.MolFromSmiles(smiles)
        except:
            mol = None
        if mol != None:
            n_atoms = mol.GetNumAtoms()
            n_atoms_list = [nat for nat in range(n_atoms)]
            if n_atoms != 0:
                for iatoms in range(n_atoms):
                    n_atoms_list_tmp = self.rotate_atoms(n_atoms_list, iatoms)  # rotate atoms' index
                    nmol = Chem.RenumberAtoms(mol, n_atoms_list_tmp)  # renumber atoms in mol
                    try:
                        smiles = Chem.MolToSmiles(nmol,
                                                  isomericSmiles=True,  # keep isomerism
                                                  kekuleSmiles=False,  # kekulize or not
                                                  rootedAtAtom=-1,  # default
                                                  canonical=False,  # canonicalize or not
                                                  allBondsExplicit=False,  #
                                                  allHsExplicit=False)  #
                    except:
                        smiles = 'None'
                    smiles_list.append(smiles)
            else:
                smiles = 'None'
                smiles_list.append(smiles)
        else:
            try:
                smiles = Chem.MolToSmiles(mol,
                                          isomericSmiles=True,  # keep isomerism
                                          kekuleSmiles=False,  # kekulize or not
                                          rootedAtAtom=-1,  # default
                                          canonical=False,  # canonicalize or not
                                          allBondsExplicit=False,  #
                                          allHsExplicit=False)  #
            except:
                smiles = 'None'
            smiles_list.append(smiles)
        smiles_array = pd.DataFrame(smiles_list).drop_duplicates().values
        # """
        if int(prop[0])==-99 and int(prop[1])==-99 and int(prop[3])==-99:
            smiles_aug = smiles_array[1:, :]
            np.random.shuffle(smiles_aug)
            smiles_array = np.vstack((smiles_array[0, :], smiles_aug[:self.aug_indicator-1, :]))
        return smiles_array

    """SMILES Augmentation"""
    def smiles_augmentation(self, df):
        column_list = df.columns
        data_aug = np.zeros((1, df.shape[1]))
        for i in range(df.shape[0]):
            smiles = df.iloc[i, 0]
            prop = df.iloc[i, 1:]
            smiles_array = self.generate_smiles(smiles,prop)
            if 'None' not in smiles_array:
                prop = np.tile(prop, (len(smiles_array), 1))
                data_new = np.hstack((smiles_array, prop))
                data_aug = np.vstack((data_aug, data_new))
        df_aug = pd.DataFrame(data_aug[1:, :], columns=column_list)
        return df_aug

In [6]:
batch_size = 24

In [7]:
class MaskedLoss(nn.Module):
    def __init__(self):
        super(MaskedLoss, self).__init__()
        
    def forward(self, outputs, props, loss_mask, properties):
        loss = {}
        for i,property in enumerate(properties):
            loss[property] = torch.sum(((outputs[:,i]-props[:,i])*loss_mask[:,i])**2.0)  / (1 if torch.sum(loss_mask[:,i])==0 else 
                                                                                            torch.sum(loss_mask[:,i]))
        return loss

In [8]:
"""MT Model"""
class MultiTask(nn.Module):
    def __init__(self, drop_rate=0.1):
        super(MultiTask, self).__init__()
        
        self.PretrainedModel = deepcopy(PretrainedModel)
        self.PretrainedModel.resize_token_embeddings(len(tokenizer))

        self.Eea_FCNN = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(768, 256),
            nn.SiLU(),
            nn.Linear(256, 64),
            nn.SiLU(),
            nn.Linear(64,1)
        )

        self.Egb_FCNN = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(768, 256),
            nn.SiLU(),
            nn.Linear(256, 64),
            nn.SiLU(),
            nn.Linear(64,1)
        )

        self.Egc_FCNN = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(768, 256),
            nn.SiLU(),
            nn.Linear(256, 64),
            nn.SiLU(),
            nn.Linear(64,1)
        )

        self.Ei_FCNN = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(768, 256),
            nn.SiLU(),
            nn.Linear(256, 64),
            nn.SiLU(),
            nn.Linear(64,1)
        )

    
    def forward(self, input_ids, attention_mask, smile_len, batch_size):
        fingerprint = torch.empty(batch_size,768)
        outputs = self.PretrainedModel(input_ids=input_ids, attention_mask=attention_mask)
        
        for i in range(batch_size):   
            fingerprint[i] = torch.mean(outputs.last_hidden_state[i, 0:smile_len[i], :],dim=0)
            
        fingerprint = (fingerprint.to(device)).double()

        Eea_FCNN_output = self.Eea_FCNN(fingerprint)
        Egb_FCNN_output = self.Egb_FCNN(fingerprint)
        Egc_FCNN_output = self.Egc_FCNN(fingerprint)
        Ei_FCNN_output = self.Ei_FCNN(fingerprint)
        
        final_output = torch.cat([Eea_FCNN_output,Egb_FCNN_output,Egc_FCNN_output,Ei_FCNN_output], dim=1)
        
        return final_output

In [9]:
def r2score(y_pred, y_actual, mask):
    bool_mask = mask.bool()
    masked_y_pred = y_pred[bool_mask]
    masked_y_actual = y_actual[bool_mask]

    ssr = torch.sum((masked_y_pred - masked_y_actual)**2)
    sst = torch.sum((masked_y_actual - torch.mean(masked_y_actual))**2)
    r2 = 1 - (ssr / sst)

    return r2

In [10]:
def train(model, optimizer, Loss, train_dataloader, device, properties):
    print("Inside Train function ")
    for step, batch in enumerate(train_dataloader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        smile_len = batch["smile_len"].to(device)
        batch_size = len(smile_len)
        loss_mask = batch["loss_mask"].to(device)
        props = batch["props"].to(device)
        outputs = model(input_ids, attention_mask, smile_len, batch_size).float()
        total_loss = 0.0
        loss = Loss(outputs, props.float(), loss_mask, properties)
        for i,property in enumerate(properties):
            total_loss += loss[property]
        print(total_loss)
#         total_loss.backward()
#         optimizer.step()
#         scheduler.step()

    return None

In [11]:
def test(model, Loss, train_dataloader, test_dataloader, device, scaler, optimizer, epoch, properties):

    train_loss = {}
    test_loss = {}
    r2_train = {}
    r2_test = {}

    for i,property in enumerate(properties):
        train_loss[property] = 0.0
        test_loss[property] = 0.0
        r2_train[property] = 0.0
        r2_test[property] = 0.0
    
    model.eval()

    with torch.no_grad():
        train_pred = {property: [] for property in properties}
        train_true = {property: [] for property in properties}
        test_pred = {property: [] for property in properties}
        test_true = {property: [] for property in properties}
        train_loss_mask = {property: [] for property in properties}
        test_loss_mask = {property: [] for property in properties}
        
        print("Inside Train_Dataloader")
        for step, batch in enumerate(train_dataloader):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            smile_len = batch["smile_len"].to(device)
            loss_mask = batch["loss_mask"].to(device)            
            props = batch["props"].to(device)
            batch_size = len(smile_len)
            outputs = model(input_ids, attention_mask, smile_len, batch_size).float()

            for i,property in enumerate(properties):
                outputs[:,i] = torch.from_numpy(scaler[i].inverse_transform(outputs[:,i].cpu()))
                props[:,i] = torch.from_numpy(scaler[i].inverse_transform(props[:,i].cpu()))
            
            loss = Loss(outputs, props.float(), loss_mask, properties)

            for i,property in enumerate(properties):
                train_loss[property] += loss[property].item() * torch.sum(loss_mask[:,i])
                train_pred[property].append(outputs[:, i].unsqueeze(1).double().to(device))
                train_true[property].append(props[:, i].unsqueeze(1).double().to(device)) 
                train_loss_mask[property].append(loss_mask[:, i].unsqueeze(1).double().to(device))
            print("End of Batch -----------------------------------------------------------------")

        for i,property in enumerate(properties):
            train_pred[property] = torch.cat(train_pred[property], dim=0)
            train_true[property] = torch.cat(train_true[property], dim=0)
            train_loss_mask[property] = torch.cat(train_loss_mask[property], dim=0)

        for i,property in enumerate(properties):    
            train_loss[property] = train_loss[property] / torch.sum(train_loss_mask[property])
            print(f'train loss for {property} after epoch is {train_loss[property]} and mask is {torch.sum(train_loss_mask[property])}')
            r2_train[property] = r2score(train_pred[property].flatten().to("cpu"), train_true[property].flatten().to("cpu"), train_loss_mask[property].flatten().to("cpu"))
            print(f'train {property} RMSE = ', torch.sqrt(train_loss[property]).item())
            print(f'train {property} r^2 = ', r2_train[property].item())
        
        print("Inside Test_Dataloader")
        for step, batch in enumerate(test_dataloader):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            smile_len = batch["smile_len"].to(device)
            loss_mask = batch["loss_mask"].to(device)            
            props = batch["props"].to(device)
            batch_size = len(smile_len)
            outputs = model(input_ids, attention_mask, smile_len, batch_size).float()

            for i,property in enumerate(properties):
                outputs[:,i] = torch.from_numpy(scaler[i].inverse_transform(outputs[:,i].cpu()))
                props[:,i] = torch.from_numpy(scaler[i].inverse_transform(props[:,i].cpu()))
            
            loss = Loss(outputs, props.float(), loss_mask, properties)
            
            for i,property in enumerate(properties):
                test_loss[property] += loss[property].item() * torch.sum(loss_mask[:,i])
                test_pred[property].append(outputs[:, i].unsqueeze(1).double().to(device))
                test_true[property].append(props[:, i].unsqueeze(1).double().to(device)) 
                test_loss_mask[property].append(loss_mask[:, i].unsqueeze(1).double().to(device))
            print('End of Batch ---------------------------------------------- -------------------------')

        for i,property in enumerate(properties):
            test_pred[property] = torch.cat(test_pred[property], dim=0)
            test_true[property] = torch.cat(test_true[property], dim=0)
            test_loss_mask[property] = torch.cat(test_loss_mask[property], dim=0)

        for i,property in enumerate(properties):    
            test_loss[property] = test_loss[property] / torch.sum(test_loss_mask[property])
            print(f'test loss for {property} after epoch is {test_loss[property]} and mask is {torch.sum(test_loss_mask[property])}')
            r2_test[property] = r2score(test_pred[property].flatten().to("cpu"), test_true[property].flatten().to("cpu"), test_loss_mask[property].flatten().to("cpu"))
            print(f'test {property} RMSE = ', torch.sqrt(test_loss[property]).item())
            print(f'test {property} r^2 = ', r2_test[property].item())

    return train_loss, test_loss, r2_train, r2_test

In [12]:
properties = ['Eea','Egb','Egc','Ei']
scaler = []
            
for i,property in enumerate(properties):
    scaled = StandardScaler()

    train_data.loc[train_data[property] != -99, property] = scaled.fit_transform(
        train_data.loc[train_data[property] != -99, [property]]).flatten()

    test_data.loc[test_data[property] != -99, property] = scaled.transform(
        test_data.loc[test_data[property] != -99, [property]]).flatten()
                
    scaler.append(scaled)
    
test_data

,smiles,Eea,Egb,Egc,Ei
2972,*Nc1ccc(C(C)(C)c2ccc(NC(=O)c3cc(C(*)=O)cc(N4C(...,-99.0,-99.0,-0.606232,-99.0
2973,*c1cccc(C(F)(F)C(F)(F)C(*)(F)F)c1,-99.0,-99.0,0.654547,-99.0
2974,*CCN(*)C(=O)c1ccc(F)cc1,-99.0,-99.0,0.407975,-99.0
2975,*c1ccc2cc(N(*)c3ccccc3)ccc2c1,-99.0,-99.0,-1.067683,-99.0
2976,*Nc1cc(C)c(*)cc1C,-99.0,-99.0,-0.380404,-99.0
...,...,...,...,...,...
3711,*CCCCCOC(=O)C(=O)O*,-99.0,-99.0,0.668569,-99.0
3712,*CCCCCCCCCCCCNC(=O)CCCCC(=O)N*,-99.0,-99.0,0.659477,-99.0
3713,*c1ccc(Oc2ccc(-c3cnc4ccc(-c5ccc6ncc(*)nc6c5)cc...,-99.0,-99.0,-0.827129,-99.0
3714,*CCOCCOC(=O)NC(=CC=C1(C))C=C1NC(=O)O*,-99.0,-99.0,0.365908,-99.0


In [13]:
print("Data Augamentation")
DataAug = MultiTask_DataAugmentation(2)
train_data = DataAug.smiles_augmentation(train_data)

"""Train the model"""
model = MultiTask(drop_rate=0.1).to(device)
model = model.double()
Loss = MaskedLoss()

optimizer = AdamW(
            [
                {"params": model.PretrainedModel.parameters(), "lr": 0.00005,
                         "weight_decay": 0.0},
                {'params': model.Eea_FCNN.parameters(), 'lr': 0.0001, 'weight_decay': 0.01},
                {'params': model.Egb_FCNN.parameters(), 'lr': 0.0001, 'weight_decay': 0.01},
                {'params': model.Egc_FCNN.parameters(), 'lr': 0.0001, 'weight_decay': 0.01},
                {'params': model.Ei_FCNN.parameters(), 'lr': 0.0001, 'weight_decay': 0.01}                     
            ],
            no_deprecation_warning=True
        )


Data Augamentation


In [14]:
train_dataset = MultiTask_Dataset(train_data,tokenizer, 411)
test_dataset = MultiTask_Dataset(test_data,tokenizer, 411)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=False, num_workers=8)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=8)

In [15]:
train_data

,smiles,Eea,Egb,Egc,Ei
0,*NC(=S)NC(=S)*,-99.0,-99.0,-1.067939,-99.0
1,*C(NC(N*)=S)=S,-99.0,-99.0,-1.067939,-99.0
2,*Oc1ccc(Oc2ccc(OC(*)=O)cc2)cc1,-99.0,-99.0,0.148853,-99.0
3,O=C(Oc1ccc(Oc2ccc(O*)cc2)cc1)*,-99.0,-99.0,0.148853,-99.0
4,*NCC(F)(F)C(*)(F)F,-1.5757,1.895784,-99.0,2.061071
...,...,...,...,...,...
12177,*c1cc(-c2cccc(C)c2)c(O*)c(-c2ccccc2)c1,-99.0,-99.0,-0.399549,-99.0
12178,*CCC(=O)OC(=O)CCC1CCC(*)O1,-99.0,-99.0,0.632714,-99.0
12179,O1C(CCC(OC(CC*)=O)=O)CCC1*,-99.0,-99.0,0.632714,-99.0
12180,*c1ccc(Cc2ccc(N3C(=O)c4ccc([Si](C)(C)c5ccc6c(c...,-99.0,-99.0,-0.987007,-99.0


In [ ]:
for epoch in range(100):
    train(model, optimizer, Loss, train_dataloader, device, properties)
    print("Starting of Test Function")
    test(model, Loss, train_dataloader, test_dataloader, device, scaler, optimizer, epoch, properties)
    print('End of Epoch//////////////////////////////////////////////////////////////////////////////////')
    print()

Inside Train function 
tensor(11.1244, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.8431, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.3439, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.3446, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.8494, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.9262, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.9428, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.2786, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0626, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.0674, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.1206, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.6735, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.9917, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.2261, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.3397, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.2488, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10.0453, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.8668, device='

tensor(1.3374, device='cuda:0', grad_fn=<AddBackward0>)
tensor(7.7393, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.5401, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.4216, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.8868, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.9299, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.2385, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.4557, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.4679, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.3281, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.8722, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9016, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7101, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.2401, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.5131, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.6906, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.4082, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.4559, device='cuda:0', grad_fn=<AddBack

tensor(2.0385, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.1060, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3493, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.3203, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.3480, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.0056, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.9828, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.5003, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.5048, device='cuda:0', grad_fn=<AddBackward0>)
tensor(7.1101, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.2021, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.4018, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.8884, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.0302, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.4093, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7728, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.8794, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7214, device='cuda:0', grad_fn=<AddBack

tensor(1.8152, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.1185, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.7085, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.6559, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.2868, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.2302, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7250, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.4908, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.2965, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5408, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.4458, device='cuda:0', grad_fn=<AddBackward0>)
tensor(7.4548, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.3066, device='cuda:0', grad_fn=<AddBackward0>)
tensor(7.3686, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.7464, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.8711, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.7110, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.5465, device='cuda:0', grad_fn=<AddBack

End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch ---------------------------------------

End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch ---------------------------------------

End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch ---------------------------------------

End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch ---------------------------------------

End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch -----------------------------------------------------------------
End of Batch ---------------------------------------